# The main goal is to locate cities with stereoptyically "ideal" weather, and a nearby hotel.

# Import Important Libraries.

In [1]:
!pip install gmaps

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import pathlib

# Import API key
from api_keys import google_map_key

# To export our map to html format
from ipywidgets.embed import embed_minimal_html

### Using os, pathlib to get the current file.


### We will delete the element of the list that represents the current name file 'starter_code' and we will replace it ### by our output_data and Images

In [3]:
current_path = pathlib.Path().resolve()
current_path

WindowsPath('C:/Users/lenovo/Desktop/06-Python-APIs (1)/06-Python-APIs/Instructions/starter_code')

#### We will split our current file to folder path and file which is the name of our current file

#### Folder_path will help us to uplod our output CSV file and images to any path we want.

In [4]:
folder_path, file = os.path.split(current_path)
folder_path

'C:\\Users\\lenovo\\Desktop\\06-Python-APIs (1)\\06-Python-APIs\\Instructions'

### Store Part I results into DataFrame

#### Load the csv exported in Part I to a DataFrame

In [5]:
cities_weather_df = pd.read_csv(os.path.join(folder_path,'output_data','Clean_City_Data.csv'))

cities_weather_df.tail()

,clean_city_data,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
568,568,Praia,14.9215,-23.5087,301.45,61,0,4.12,CV,1654969216
569,569,Utica,43.1009,-75.2327,297.79,52,83,2.61,US,1654969217
570,570,Diamantina,-18.2494,-43.6003,297.73,36,57,1.76,BR,1654969217
571,571,Tooele,40.5308,-112.2983,304.83,28,81,2.59,US,1654969218
572,572,Marshall,34.3834,-86.3333,303.75,54,40,0.45,US,1654969219


In [6]:
len(cities_weather_df)

573

### Clean our DataFrame if there's any non significant values can reduce the fiability of our Data

In [7]:
cities_weather_df = cities_weather_df.dropna()

## Humidity Heatmap
#### . Configure gmaps.
#### . Use the Lat and Lng as locations and Humidity as the weight.
#### . Add Heatmap layer to map.

In [8]:
#the locations and weight for the map
locations = cities_weather_df[ ['Latitude','Longitude'] ].astype(float)
humidity_rate  = cities_weather_df['Humidity'].astype(float)

In [9]:
# Configure gmaps with API key
gmaps.configure(api_key = google_map_key)

In [10]:
# Create a poverty Heatmap layer
fig = gmaps.figure(center=(48.0, 0.0), zoom_level=3)

heat_layer = gmaps.heatmap_layer(locations, weights = humidity_rate, 
                                 dissipating = False, max_intensity = 150,
                                 point_radius = 3, opacity = 1)
fig.add_layer(heat_layer)

#### Show the Map Figure of our Locations

In [11]:
fig

Figure(layout=FigureLayout(height='420px'))

#### Save the Map figure as PNG

#### At the moment, exporting maps that contain a directions layer to PNG is not supported (because html2canvas 0.4.1 doesn't support it).

## Solution to import the Map Image

#### You can export maps to HTML using the infrastructure provided by ipywidgets. For instance, let’s export a simple map to HTML:

In [12]:
!pip install ipywidgets

In [13]:
embed_minimal_html(os.path.join(folder_path,'Images','heat_map.html'), views=[fig])

## Create new DataFrame fitting weather criteria
#### Narrow down the cities to fit weather conditions.
#### Drop any rows will null values.

In [14]:
# Filter and Copy the DataFrame
#use loc statements to narrow down the resulys

vacation_cities = cities_weather_df.loc[
                (cities_weather_df['Max Temperature'] < 80) & (cities_weather_df['Max Temperature'] > 70) &
                (cities_weather_df['Wind Speed'] < 10) & (cities_weather_df['Cloudiness'] == 0)
                ] 

In [15]:
vacation_cities.head()

,clean_city_data,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date


## Hotel Map
#### Store into variable named hotel_df.
#### Add a "Hotel Name" column to the DataFrame.
#### Set parameters to search for hotels with 5000 meters.
#### Hit the Google Places API for each city's coordinates.
#### Store the first Hotel result into the DataFrame.
#### Plot markers on top of the heatmap.

In [16]:
#Store into variable named hotel_df.
hotel_df = vacation_cities.copy()

hotel_df.head()

,clean_city_data,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date


In [17]:
#Add a "Hotel Name" column to the DataFrame.

hotel_df["Hotel Name"] = ""

In [18]:
hotel_df.head()

,clean_city_data,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name


In [19]:
radius = 5000
gtype = "hotels"

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"radius":radius, "types": gtype, "key":google_map_key}

for index, row in hotel_df.iterrows():
    city = row['City']
    latitude = row['Latitude']
    longitude = row['Longitude']

    location = row[["Latitude", "Longitude"]].values
    
    params["location"]= f'{location[0]},{location[1]}'
    
    query_url = requests.get(base_url, params=params).json()
    results = query_url['results']
    
    try:
        hotelname = results[0]['name']
        hotel_df.loc[index, "Hotel Name"] = hotelname
        print(f"Closest hotel to {city} at {latitude} , {longitude} is {hotelname}.")
    except (KeyError, IndexError):
        print("There is a missing field. Moving past.....")

In [20]:
#check if the hotels got added
hotel_df.head()

,clean_city_data,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name


In [21]:
# Get all The Hotel names that are not empty
#hotel_df=hotel_df[hotel_df["Hotel Name"]!= ""] 

In [22]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

In [23]:
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations_2 = hotel_df[["Latitude", "Longitude"]]

In [24]:
fig_markers_layer = gmaps.figure(center=(48.0, 0.0), zoom_level=2)

marker_layer = gmaps.marker_layer(locations_2, hover_text='', label='', info_box_content = hotel_info)

fig_markers_layer.add_layer(marker_layer)

In [25]:
# Display figure
fig_markers_layer

Figure(layout=FigureLayout(height='420px'))

In [26]:
embed_minimal_html(os.path.join(folder_path,'Images','hotel_map.html'), views=[fig_markers_layer])